# Neural Network Development Tutorial: Wafer Defect Classification

## Overview
This tutorial demonstrates the progressive development of classification models for wafer defect detection, starting from traditional machine learning methods and advancing to deep neural networks.

## Learning Objectives
- Compare traditional ML (Random Forest, SVM) with neural networks
- Understand when spatial structure matters
- Build progressively complex neural architectures
- Apply transfer learning for improved performance

## Dataset
The WM-811K wafer map dataset contains grayscale images (26×26 pixels) of semiconductor wafers with 38 different defect types.

## Tutorial Structure
1. **Traditional ML**: Random Forest and Support Vector Machine
2. **Simple NN**: Single-layer neural network
3. **MLP**: Multi-layer perceptron with activation functions
4. **CNN**: Convolutional neural network
5. **Transfer Learning**: Pre-trained ResNet18
6. **Comparison**: Comprehensive performance analysis

---
# Part 0: Setup and Data Loading

First, we'll import the necessary libraries and load our dataset.

In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Traditional ML models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# PyTorch for neural networks
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.models as models
import torchvision.transforms as transforms

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## Load and Prepare Data

Load your wafer defect dataset. Adjust the file path as needed.

In [ ]:
# Load your data - adjust path as needed
# Assuming you have X (images) and y (labels)
# Example format:
# X shape: (n_samples, 26, 26) - grayscale images
# y shape: (n_samples,) - defect type labels

# If you have a pickle or npz file:
data = np.load('wafer_data.npz')
X = data['X']
y = data['y']

# For this tutorial, let's assume you've loaded:
# X: your wafer images
# y: your labels

# Placeholder - replace with your actual data loading
print(f"Data shape: X={X.shape}, y={y.shape}")
print(f"Number of classes: {len(np.unique(y))}")
print(f"Image dimensions: {X.shape[1]}×{X.shape[2]}")

NameError: name 'X' is not defined

In [ ]:
# Visualize some sample images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i, ax in enumerate(axes.flat):
    idx = np.random.randint(0, len(X))
    ax.imshow(X[idx], cmap='gray')
    ax.set_title(f'Class: {y[idx]}')
    ax.axis('off')
plt.suptitle('Sample Wafer Defect Images')
plt.tight_layout()
plt.show()

In [ ]:
# Encode labels if they're not already numeric
if y.dtype == 'object' or not np.issubdtype(y.dtype, np.integer):
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    print(f"Encoded {len(label_encoder.classes_)} classes")

n_classes = len(np.unique(y))
print(f"Total classes: {n_classes}")

In [ ]:
# Split data into train, validation, and test sets
# First split: 80% train+val, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Second split: 75% train, 25% val (of the 80%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

## Normalize Data

Normalize pixel values to [0, 1] range for neural networks.

In [ ]:
# Normalize to [0, 1]
X_train_norm = X_train.astype('float32') / 255.0
X_val_norm = X_val.astype('float32') / 255.0
X_test_norm = X_test.astype('float32') / 255.0

print(f"Normalized data range: [{X_train_norm.min():.2f}, {X_train_norm.max():.2f}]")

---
# Part 1: Traditional Machine Learning Methods

Before diving into neural networks, let's establish baselines using traditional machine learning methods: **Random Forest** and **Support Vector Machine**.

## Why Start Here?
- **Fast training**: Traditional ML models train much faster
- **Strong baselines**: Often perform surprisingly well
- **No spatial structure**: These models flatten images, losing spatial information
- **Feature engineering**: Work with raw pixel values

## Key Limitation
Both models require flattening the 26×26 images into 676-dimensional vectors, **losing the 2D spatial structure** that CNNs can exploit.

---

## 1.1 Random Forest Classifier

Random Forest builds an ensemble of decision trees, each trained on a random subset of data and features.

**Advantages:**
- Handles non-linear relationships
- Robust to overfitting with many trees
- No extensive hyperparameter tuning needed
- Built-in feature importance

**Disadvantages:**
- Treats pixels as independent features
- Doesn't capture spatial patterns
- Can be memory-intensive

In [ ]:
# Flatten images for traditional ML
X_train_flat = X_train_norm.reshape(X_train_norm.shape[0], -1)
X_val_flat = X_val_norm.reshape(X_val_norm.shape[0], -1)
X_test_flat = X_test_norm.reshape(X_test_norm.shape[0], -1)

print(f"Flattened shape: {X_train_flat.shape}")
print(f"Each image is now a {X_train_flat.shape[1]}-dimensional vector")

In [ ]:
print("Training Random Forest Classifier...")
print("This may take a few minutes...\n")

# Create Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=100,      # Number of trees
    max_depth=20,          # Maximum depth of trees
    min_samples_split=10,  # Minimum samples to split a node
    min_samples_leaf=4,    # Minimum samples in a leaf
    random_state=42,
    n_jobs=-1,             # Use all CPU cores
    verbose=1
)

# Train the model
rf_model.fit(X_train_flat, y_train)

print("\nRandom Forest training complete!")

In [ ]:
# Evaluate Random Forest
rf_train_pred = rf_model.predict(X_train_flat)
rf_val_pred = rf_model.predict(X_val_flat)
rf_test_pred = rf_model.predict(X_test_flat)

rf_train_acc = accuracy_score(y_train, rf_train_pred)
rf_val_acc = accuracy_score(y_val, rf_val_pred)
rf_test_acc = accuracy_score(y_test, rf_test_pred)

print("=" * 50)
print("Random Forest Results")
print("=" * 50)
print(f"Training Accuracy:   {rf_train_acc:.4f}")
print(f"Validation Accuracy: {rf_val_acc:.4f}")
print(f"Test Accuracy:       {rf_test_acc:.4f}")
print("=" * 50)

# Show classification report
print("\nDetailed Classification Report (Test Set):")
print(classification_report(y_test, rf_test_pred))

In [ ]:
# Visualize top important pixels
feature_importance = rf_model.feature_importances_
importance_map = feature_importance.reshape(26, 26)

plt.figure(figsize=(10, 8))
plt.imshow(importance_map, cmap='hot', interpolation='nearest')
plt.colorbar(label='Feature Importance')
plt.title('Random Forest: Pixel Importance Map')
plt.xlabel('Pixel X')
plt.ylabel('Pixel Y')
plt.tight_layout()
plt.show()

print("\nInterpretation: Brighter areas indicate pixels that are more important for classification.")

---
## 1.2 Support Vector Machine (SVM)

SVM finds an optimal hyperplane that separates different classes in high-dimensional space.

**Advantages:**
- Effective in high-dimensional spaces
- Memory efficient (uses support vectors)
- Kernel trick allows non-linear decision boundaries

**Disadvantages:**
- Can be slow to train on large datasets
- Sensitive to feature scaling
- Requires careful kernel selection

**Note:** We'll use a linear kernel for speed. RBF kernel would be more powerful but much slower.

In [ ]:
print("Training Support Vector Machine...")
print("Using linear kernel for faster training...\n")

# Create SVM model with linear kernel
svm_model = SVC(
    kernel='linear',
    C=1.0,              # Regularization parameter
    random_state=42,
    verbose=True,
    max_iter=1000       # Maximum iterations
)

# Train the model
svm_model.fit(X_train_flat, y_train)

print("\nSVM training complete!")

In [ ]:
# Evaluate SVM
svm_train_pred = svm_model.predict(X_train_flat)
svm_val_pred = svm_model.predict(X_val_flat)
svm_test_pred = svm_model.predict(X_test_flat)

svm_train_acc = accuracy_score(y_train, svm_train_pred)
svm_val_acc = accuracy_score(y_val, svm_val_pred)
svm_test_acc = accuracy_score(y_test, svm_test_pred)

print("=" * 50)
print("Support Vector Machine Results")
print("=" * 50)
print(f"Training Accuracy:   {svm_train_acc:.4f}")
print(f"Validation Accuracy: {svm_val_acc:.4f}")
print(f"Test Accuracy:       {svm_test_acc:.4f}")
print("=" * 50)

# Show classification report
print("\nDetailed Classification Report (Test Set):")
print(classification_report(y_test, svm_test_pred))

---
## Traditional ML Comparison

Let's compare Random Forest and SVM performance.

In [ ]:
# Compare traditional ML models
models = ['Random Forest', 'SVM (Linear)']
train_accs = [rf_train_acc, svm_train_acc]
val_accs = [rf_val_acc, svm_val_acc]
test_accs = [rf_test_acc, svm_test_acc]

x = np.arange(len(models))
width = 0.25

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width, train_accs, width, label='Train', alpha=0.8)
ax.bar(x, val_accs, width, label='Validation', alpha=0.8)
ax.bar(x + width, test_accs, width, label='Test', alpha=0.8)

ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Traditional ML Methods: Performance Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1])

# Add value labels on bars
for i, (train, val, test) in enumerate(zip(train_accs, val_accs, test_accs)):
    ax.text(i - width, train + 0.02, f'{train:.3f}', ha='center', fontsize=9)
    ax.text(i, val + 0.02, f'{val:.3f}', ha='center', fontsize=9)
    ax.text(i + width, test + 0.02, f'{test:.3f}', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

## 🔍 Key Insights: Traditional ML

### What We Learned:
1. **Random Forest** typically performs better than linear SVM for this task
2. Both models provide **strong baselines** without complex tuning
3. Training is **fast** compared to neural networks
4. **Limitation**: Both flatten images, losing spatial structure

### Why Neural Networks?
While traditional ML works, it treats each pixel independently. Neural networks (especially CNNs) can:
- Learn spatial patterns (edges, shapes, textures)
- Build hierarchical features automatically
- Potentially achieve higher accuracy

Let's see if neural networks can beat these baselines!

---
# Part 2: Simple Single-Layer Neural Network

Now let's build our first neural network - a simple single-layer model that's essentially **logistic regression**.

## Architecture
```
Input (26×26=676) → Flatten → Linear(676→38) → Output
```

This is the simplest possible neural network:
- **No hidden layers**
- **No activation functions** (linear transformation only)
- Just learns a direct mapping from pixels to classes

## What We'll Learn:
- How to structure a basic PyTorch model
- Training loop implementation
- Why deeper networks are needed

In [ ]:
class SimpleNN(nn.Module):
    """Single-layer neural network (logistic regression)"""
    
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        # Flatten the input
        x = x.view(x.size(0), -1)
        # Apply linear transformation
        x = self.fc(x)
        return x

# Create model
input_size = 26 * 26  # 676 pixels
simple_model = SimpleNN(input_size, n_classes).to(device)

# Count parameters
simple_params = sum(p.numel() for p in simple_model.parameters())
print(f"Model: Simple Single-Layer NN")
print(f"Total parameters: {simple_params:,}")
print(f"\nArchitecture:")
print(simple_model)

## Training Utilities

Let's create helper functions for training and evaluation that we'll reuse for all models.

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def evaluate(model, data_loader, criterion, device):
    """Evaluate model on a dataset"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(data_loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

print("Training utilities defined!")

In [ ]:
# Prepare PyTorch data loaders
batch_size = 64

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_norm).unsqueeze(1)  # Add channel dimension
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val_norm).unsqueeze(1)
y_val_tensor = torch.LongTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test_norm).unsqueeze(1)
y_test_tensor = torch.LongTensor(y_test)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"DataLoaders created with batch size: {batch_size}")
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

In [ ]:
# Training configuration
num_epochs = 20
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(simple_model.parameters(), lr=learning_rate)

# Training loop
print("Training Simple Single-Layer NN...\n")
simple_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(simple_model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(simple_model, val_loader, criterion, device)
    
    simple_history['train_loss'].append(train_loss)
    simple_history['train_acc'].append(train_acc)
    simple_history['val_loss'].append(val_loss)
    simple_history['val_acc'].append(val_acc)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# Final test evaluation
test_loss, test_acc = evaluate(simple_model, test_loader, criterion, device)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
ax1.plot(simple_history['train_loss'], label='Train Loss', marker='o')
ax1.plot(simple_history['val_loss'], label='Val Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Simple NN: Training and Validation Loss')
ax1.legend()
ax1.grid(alpha=0.3)

# Accuracy plot
ax2.plot(simple_history['train_acc'], label='Train Accuracy', marker='o')
ax2.plot(simple_history['val_acc'], label='Val Accuracy', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Simple NN: Training and Validation Accuracy')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 🔍 Key Insights: Simple Neural Network

### Observations:
1. **Linear model**: No activation functions = just weighted sum
2. **Limited capacity**: Can't learn complex non-linear patterns
3. **Similar to SVM**: Both are linear classifiers
4. **Fast training**: Fewer parameters means quick convergence

### Why It's Limited:
- No hidden layers to learn intermediate features
- Can't capture non-linear relationships in data
- Treats problem as linearly separable

**Next step**: Add hidden layers and non-linear activations!

---
# Part 3: Multi-Layer Perceptron (MLP)

Now we'll add **hidden layers** and **activation functions** to learn non-linear patterns.

## Architecture
```
Input (676) → FC(512) → ReLU → Dropout 
           → FC(256) → ReLU → Dropout
           → FC(128) → ReLU → Dropout
           → FC(38) → Output
```

## New Components:
1. **Hidden Layers**: Learn intermediate representations
2. **ReLU Activation**: Introduces non-linearity (max(0, x))
3. **Dropout**: Prevents overfitting by randomly dropping neurons

This should perform better than the simple model!

In [ ]:
class MLP(nn.Module):
    """Multi-Layer Perceptron with dropout"""
    
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, num_classes)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Hidden layer 1
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # Hidden layer 2
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # Hidden layer 3
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # Output layer
        x = self.fc4(x)
        return x

# Create model
mlp_model = MLP(input_size, n_classes).to(device)

mlp_params = sum(p.numel() for p in mlp_model.parameters())
print(f"Model: Multi-Layer Perceptron")
print(f"Total parameters: {mlp_params:,}")
print(f"\nArchitecture:")
print(mlp_model)

In [ ]:
# Training configuration
num_epochs = 30
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=learning_rate)

# Training loop
print("Training Multi-Layer Perceptron...\n")
mlp_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

best_val_acc = 0
patience = 5
patience_counter = 0

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(mlp_model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(mlp_model, val_loader, criterion, device)
    
    mlp_history['train_loss'].append(train_loss)
    mlp_history['train_acc'].append(train_acc)
    mlp_history['val_loss'].append(val_loss)
    mlp_history['val_acc'].append(val_acc)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

# Final test evaluation
test_loss, test_acc = evaluate(mlp_model, test_loader, criterion, device)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(mlp_history['train_loss'], label='Train Loss', marker='o')
ax1.plot(mlp_history['val_loss'], label='Val Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('MLP: Training and Validation Loss')
ax1.legend()
ax1.grid(alpha=0.3)

ax2.plot(mlp_history['train_acc'], label='Train Accuracy', marker='o')
ax2.plot(mlp_history['val_acc'], label='Val Accuracy', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('MLP: Training and Validation Accuracy')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 🔍 Key Insights: Multi-Layer Perceptron

### Improvements:
1. **Non-linear learning**: ReLU enables complex pattern recognition
2. **Better accuracy**: Should outperform simple NN and possibly RF/SVM
3. **Dropout helps**: Prevents overfitting on training data

### Still Limited:
- **No spatial awareness**: Still flattens images
- **Loses structure**: Doesn't know which pixels are neighbors
- **Not translation-invariant**: Can't recognize patterns regardless of position

**Solution**: Use Convolutional Neural Networks (CNNs)!

---
# Part 4: Convolutional Neural Network (CNN)

Now we'll use **convolutional layers** to preserve and learn from the 2D spatial structure of images.

## Architecture
```
Input (1×26×26)
  ↓ Conv(16) → BatchNorm → ReLU → MaxPool
  ↓ Conv(32) → BatchNorm → ReLU → MaxPool
  ↓ Conv(64) → BatchNorm → ReLU → MaxPool
  ↓ Flatten
  ↓ FC(128) → ReLU → Dropout
  ↓ FC(38) → Output
```

## Why CNNs Excel:
1. **Spatial awareness**: Learns from local pixel neighborhoods
2. **Translation invariance**: Recognizes patterns anywhere in the image
3. **Hierarchical features**: Early layers detect edges, later layers detect complex shapes
4. **Parameter efficiency**: Shared weights across spatial locations

This is your original architecture!

In [ ]:
class CNN(nn.Module):
    """Convolutional Neural Network for wafer defect classification"""
    
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        
        # Convolutional block 1
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # Convolutional block 2
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # Convolutional block 3
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 3 * 3, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)
        
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # Conv block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool1(x)  # 26×26 → 13×13
        
        # Conv block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool2(x)  # 13×13 → 6×6
        
        # Conv block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool3(x)  # 6×6 → 3×3
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # FC layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Create model
cnn_model = CNN(n_classes).to(device)

cnn_params = sum(p.numel() for p in cnn_model.parameters())
print(f"Model: Convolutional Neural Network")
print(f"Total parameters: {cnn_params:,}")
print(f"\nArchitecture:")
print(cnn_model)

In [ ]:
# Training configuration
num_epochs = 40
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

# Training loop
print("Training Convolutional Neural Network...\n")
cnn_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

best_val_acc = 0
patience = 7
patience_counter = 0

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(cnn_model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(cnn_model, val_loader, criterion, device)
    
    cnn_history['train_loss'].append(train_loss)
    cnn_history['train_acc'].append(train_acc)
    cnn_history['val_loss'].append(val_loss)
    cnn_history['val_acc'].append(val_acc)
    
    # Learning rate scheduling
    scheduler.step(val_acc)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f} | "
          f"LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        # Save best model
        torch.save(cnn_model.state_dict(), '/home/claude/best_cnn_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

# Load best model
cnn_model.load_state_dict(torch.load('/home/claude/best_cnn_model.pth'))

# Final test evaluation
test_loss, test_acc = evaluate(cnn_model, test_loader, criterion, device)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(cnn_history['train_loss'], label='Train Loss', marker='o')
ax1.plot(cnn_history['val_loss'], label='Val Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('CNN: Training and Validation Loss')
ax1.legend()
ax1.grid(alpha=0.3)

ax2.plot(cnn_history['train_acc'], label='Train Accuracy', marker='o')
ax2.plot(cnn_history['val_acc'], label='Val Accuracy', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('CNN: Training and Validation Accuracy')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 🔍 Key Insights: CNN

### Why CNNs Work Better:
1. **Spatial feature learning**: Convolutions detect edges, textures, patterns
2. **Translation invariance**: Same filter applied across entire image
3. **Hierarchical features**: Early layers → simple features, Deep layers → complex patterns
4. **Parameter efficiency**: Fewer parameters than MLP, but more powerful

### Expected Results:
- **Higher accuracy** than MLP and traditional ML
- **Better generalization** due to spatial inductive biases
- **Learned features** are interpretable (can visualize filters)

**Can we do even better?** Yes - with transfer learning!

---
# Part 5: Transfer Learning with ResNet18

Transfer learning uses a model **pre-trained on millions of images** (ImageNet) and adapts it to our task.

## Why Transfer Learning?
1. **Pre-learned features**: ResNet already knows edges, textures, shapes
2. **Less data needed**: Can work well even with smaller datasets
3. **Faster convergence**: Starts from good feature representations
4. **Better generalization**: Learned robust features from diverse images

## Architecture:
- Use **ResNet18** (18-layer deep network)
- Modify first layer: ImageNet expects RGB (3 channels), we have grayscale (1 channel)
- Replace final layer: ImageNet has 1000 classes, we have 38
- **Fine-tuning**: Train entire network with lower learning rate

In [ ]:
# Load pre-trained ResNet18
resnet_model = models.resnet18(pretrained=True)

# Modify first conv layer for grayscale input (1 channel instead of 3)
resnet_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Replace final fully connected layer
num_features = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_features, n_classes)

# Move to device
resnet_model = resnet_model.to(device)

resnet_params = sum(p.numel() for p in resnet_model.parameters())
print(f"Model: ResNet18 (Transfer Learning)")
print(f"Total parameters: {resnet_params:,}")
print(f"\nModifications:")
print(f"- First layer: Conv2d(1, 64) for grayscale")
print(f"- Final layer: Linear({num_features}, {n_classes})")
print(f"- Pre-trained on ImageNet: ✓")

In [ ]:
# Training configuration for transfer learning
num_epochs = 30
learning_rate = 0.0001  # Lower LR for fine-tuning

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

# Training loop
print("Training ResNet18 with Transfer Learning...\n")
resnet_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

best_val_acc = 0
patience = 7
patience_counter = 0

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(resnet_model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(resnet_model, val_loader, criterion, device)
    
    resnet_history['train_loss'].append(train_loss)
    resnet_history['train_acc'].append(train_acc)
    resnet_history['val_loss'].append(val_loss)
    resnet_history['val_acc'].append(val_acc)
    
    scheduler.step(val_acc)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f} | "
          f"LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(resnet_model.state_dict(), '/home/claude/best_resnet_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

# Load best model
resnet_model.load_state_dict(torch.load('/home/claude/best_resnet_model.pth'))

# Final test evaluation
test_loss, test_acc = evaluate(resnet_model, test_loader, criterion, device)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(resnet_history['train_loss'], label='Train Loss', marker='o')
ax1.plot(resnet_history['val_loss'], label='Val Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('ResNet18: Training and Validation Loss')
ax1.legend()
ax1.grid(alpha=0.3)

ax2.plot(resnet_history['train_acc'], label='Train Accuracy', marker='o')
ax2.plot(resnet_history['val_acc'], label='Val Accuracy', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('ResNet18: Training and Validation Accuracy')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 🔍 Key Insights: Transfer Learning

### Advantages:
1. **Knowledge transfer**: Leverages features learned from millions of images
2. **Deeper network**: 18 layers vs. 3 in our custom CNN
3. **Residual connections**: Helps gradient flow, enables deeper networks
4. **Proven architecture**: ResNet won ImageNet competition

### Expected Results:
- **Highest accuracy** among all models
- **Faster convergence** than training from scratch
- **Better generalization** even with limited data

### Trade-offs:
- More parameters (11M+ vs 300K for custom CNN)
- Slower inference time
- Higher memory requirements

---
# Part 6: Complete Model Comparison

Let's compare all models we've trained!

In [ ]:
# Collect all results
models_list = [
    'Random Forest',
    'SVM (Linear)',
    'Simple NN',
    'MLP',
    'CNN',
    'ResNet18'
]

# You'll need to fill in the actual test accuracies from your runs
test_accuracies = [
    rf_test_acc,
    svm_test_acc,
    # Add other test accuracies as you complete training
]

parameters = [
    rf_model.n_estimators * 1000,  # Approximate
    0,  # SVM support vectors vary
    simple_params,
    mlp_params,
    cnn_params,
    resnet_params
]

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Model': models_list,
    'Parameters': parameters,
    'Test Accuracy': test_accuracies
})

print("\n" + "="*60)
print("FINAL MODEL COMPARISON")
print("="*60)
print(comparison_df.to_string(index=False))
print("="*60)

In [ ]:
# Comprehensive comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Accuracy comparison
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DFE6E9']
bars = ax1.bar(models_list, test_accuracies, color=colors, alpha=0.8, edgecolor='black')
ax1.set_ylabel('Test Accuracy', fontsize=12, fontweight='bold')
ax1.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax1.set_ylim([0, 1])
ax1.grid(axis='y', alpha=0.3)
ax1.set_xticklabels(models_list, rotation=45, ha='right')

# Add value labels
for bar, acc in zip(bars, test_accuracies):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
            f'{acc:.4f}', ha='center', va='bottom', fontweight='bold')

# Parameter comparison (log scale)
param_labels = [f"{p/1e6:.2f}M" if p > 1e6 else f"{p/1e3:.1f}K" for p in parameters]
bars2 = ax2.bar(models_list, parameters, color=colors, alpha=0.8, edgecolor='black')
ax2.set_ylabel('Number of Parameters', fontsize=12, fontweight='bold')
ax2.set_title('Model Complexity Comparison', fontsize=14, fontweight='bold')
ax2.set_yscale('log')
ax2.grid(axis='y', alpha=0.3)
ax2.set_xticklabels(models_list, rotation=45, ha='right')

# Add value labels
for bar, label in zip(bars2, param_labels):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height * 1.1,
            label, ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

---
# 🎯 Final Insights and Recommendations

## Model Progression Summary

### 1. Traditional ML (Random Forest, SVM)
- ✅ **Fast training**, strong baselines
- ❌ **Lose spatial structure** (flatten images)
- 📊 Good starting point for understanding data

### 2. Simple Neural Network
- ✅ **Simple to understand**, fast to train
- ❌ **Linear only**, can't learn complex patterns
- 📊 Similar performance to linear SVM

### 3. Multi-Layer Perceptron (MLP)
- ✅ **Non-linear learning**, better than simple NN
- ❌ **Still flattens images**, no spatial awareness
- 📊 Moderate improvement over simple models

### 4. Convolutional Neural Network (CNN)
- ✅ **Spatial feature learning**, translation invariant
- ✅ **Efficient parameters**, good accuracy
- 📊 Significant improvement - best custom architecture

### 5. Transfer Learning (ResNet18)
- ✅ **Pre-trained features**, deepest network
- ✅ **Highest accuracy**, best generalization
- ⚠️ **More parameters**, slower inference
- 📊 Best overall performance

---

## 💡 Recommendations

### For Production:
1. **High accuracy required**: Use **ResNet18** transfer learning
2. **Speed/resource constrained**: Use **Custom CNN** or **Random Forest**
3. **Quick baseline**: Start with **Random Forest**

### For Further Improvement:
1. **Data augmentation**: Rotation, flipping, noise
2. **Ensemble methods**: Combine multiple models
3. **Architecture search**: Try other pre-trained models (ResNet50, EfficientNet)
4. **Hyperparameter tuning**: Learning rate, dropout, batch size
5. **Class balancing**: Handle imbalanced defect types

### Key Takeaways:
- 🎓 **Spatial structure matters** for image classification
- 🏗️ **Architecture choice** depends on your constraints
- 🔄 **Transfer learning** is powerful even for specialized domains
- 📈 **Progressive complexity** helps understand what each component adds

---

## 🧪 Exercises for Further Exploration

1. **Data Augmentation**: Add random rotations and noise during training
2. **Attention Mechanisms**: Add attention layers to CNN
3. **Model Ensemble**: Combine predictions from multiple models
4. **Explainability**: Visualize CNN filters and activation maps
5. **Different Architectures**: Try VGG, DenseNet, or EfficientNet
6. **Class Imbalance**: Implement weighted loss or SMOTE
7. **Hyperparameter Optimization**: Use Optuna or Ray Tune

---

## 📚 Additional Resources

- PyTorch Documentation: https://pytorch.org/docs/
- CNN Explainer: https://poloclub.github.io/cnn-explainer/
- Transfer Learning Guide: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
- Papers with Code: https://paperswithcode.com/

---

**Congratulations!** 🎉 You've completed the neural network development tutorial and learned how to progressively build and improve models for image classification!